In [5]:
import os
import tensorflow as tf
import IPython.display as display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import PIL

mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

In [6]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [7]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3, expand_animations=False)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:2], tf.float32)
  long_dim = tf.reduce_max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [38]:
def load_img_fixed(path_to_img, size=256):
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3, expand_animations=False)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, [size, size])
  img = img[tf.newaxis, :]
  return img


def get_style_embeddings(style_paths):
    embeddings = []
    for path in style_paths:
        try:
            img = load_img_fixed(path) # Ensure this returns a float32 tensor [1, H, W, 3]
            embeddings.append(img)
        except Exception as e:
            print(f"Error loading style image {path}: {e}")
            continue
    return embeddings

In [20]:
content_paths = [f"data/content/{f}" for f in os.listdir("data/content")]
style_paths = [f"data/style/{f}" for f in os.listdir("data/style")]

In [ ]:
import tensorflow_hub as hub

num = 0

for style_path in style_paths:
    hub_model = hub.load('models/arbitrary-image-stylization-v1-tensorflow1-256-v2')
    style_img = load_img(style_path)
    content_img = load_img(content_paths[0])
    stylized_image = hub_model(tf.constant(content_img), tf.constant(style_img))[0]
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"all_styles_grid2/{num}.png")
    num += 1

Error loading style image data/style/.DS_Store: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} data/style/.DS_Store; No such file or directory [Op:ReadFile]


2026-01-01 15:49:53.750312: W tensorflow/core/framework/op_kernel.cc:1855] OP_REQUIRES failed at whole_file_read_ops.cc:117 : NOT_FOUND: data/style/.DS_Store; No such file or directory
2026-01-01 15:49:53.750375: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: NOT_FOUND: data/style/.DS_Store; No such file or directory


In [ ]:
# RANDOM STYLE
import tensorflow_hub as hub

animation_frames = [f"data/animation/{f}" for f in os.listdir("data/animation")]
style_paths = [f"data/style/{f}" for f in os.listdir("data/style")]

num = 0

for frame in animation_frames:
    hub_model = hub.load('models/arbitrary-image-stylization-v1-tensorflow1-256-v2')
    random_style_int = np.random.randint(0, len(style_paths))
    try:
        style_img = load_img(style_paths[random_style_int])
    except Exception as e:
        print(f"Error loading style image {style_path}: {e}")
        continue
    content_img = load_img(frame)
    stylized_image = hub_model(tf.constant(content_img), tf.constant(style_img))[0]
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"outputs/cube_animation_1/{num}.png")
    num += 1

In [39]:
# AVERAGE STYLE
import glob
import tensorflow_hub as hub

animation_frames = sorted(glob.glob('data/animation/*.png'))
style_paths = [f"data/style2/{f}" for f in os.listdir("data/style2")]

style_embeddings = get_style_embeddings(style_paths)
avg_style = tf.reduce_mean(tf.concat(style_embeddings, axis=0), axis=0, keepdims=True)

num = 0

for frame in animation_frames:
    hub_model = hub.load('models/arbitrary-image-stylization-v1-tensorflow1-256-v2')
    random_style_int = np.random.randint(0, len(style_paths))
    stylized_image = hub_model(tf.constant(content_img), tf.constant(avg_style))[0]
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"outputs/cube_animation_2/{num}.png")
    num += 1

In [ ]:
# TRANSITIONING STYLES

import numpy as np
import tensorflow as tf

# Load model once
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def get_style_vector(path):
    # This model treats the style image input as the source for the bottleneck
    return load_img(path)

# Pre-load your style images
style_tensors = [get_style_vector(p) for p in style_paths]
num_frames = len(animation_frames)
num_styles = len(style_tensors)

for i, frame_path in enumerate(animation_frames):
    # Determine which two styles we are between
    # This maps the current frame index to a float position in the style list
    style_idx_float = (i / (num_frames - 1)) * (num_styles - 1)
    idx1 = int(np.floor(style_idx_float))
    idx2 = int(np.ceil(style_idx_float))
    
    # Calculate the blend weight (0.0 to 1.0)
    fraction = style_idx_float - idx1
    
    # Interpolate between the two style images
    # This creates a 'morphed' style image for this specific frame
    interpolated_style = (1 - fraction) * style_tensors[idx1] + fraction * style_tensors[idx2]
    
    # Run Stylization
    content_img = load_img(frame_path)
    stylized_image = hub_model(tf.constant(content_img), tf.constant(interpolated_style))[0]
    
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"outputs/cube_animation_3/{i:04d}.png")

In [43]:
#SINGLE STYLE

content_path = "data/content/grid1.jpg"
style_path = "data/style/pomelo.jpeg"

content_img = load_img(content_path)
style_img = load_img(style_path)

# Find the next available number
import re
existing = [f for f in os.listdir("outputs") if re.match(r'^\d+\.png$', f)]
if existing:
    nums = [int(re.match(r'^(\d+)\.png$', f).group(1)) for f in existing]
    num = max(nums) + 1
else:
    num = 1

stylized_image = hub_model(tf.constant(content_img), tf.constant(style_img))[0]

# Save the stylized image
output_image = tensor_to_image(stylized_image)
output_image.save(f"outputs/{num}.png")
print(f"Saved to outputs/{num}.png")

Saved to outputs/4.png
